In [4]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
from datetime import datetime
from random import sample

In [19]:
#read in stock data
tickers = ['TLT', 'IVV']
start = datetime(2003,1,1)
end = datetime(2020,7,31)
stocks=pd.DataFrame()
stock_div=pd.DataFrame()
for tick in tickers:
    stocks[tick] = data.DataReader(tick, 'yahoo', start, end)["Close"]
    stock_div[tick+' div'] = data.DataReader(tick, 'yahoo-dividends', start, end)["value"]
stocks = pd.concat([stocks,stock_div],axis=1)
stocks = stocks.fillna(0)

In [20]:
#Adding dividends to price appreciation on a daily percent return basis
raw_returns=pd.DataFrame()
for tick in tickers:
    raw_returns[tick] = stocks[tick].pct_change()+stocks[tick+' div']/stocks[tick]
returns=raw_returns[1:]
returns = returns[np.isfinite(returns).all(1)]

In [21]:
#Problem set up: couple expecting to live 10 more years provides living expenses, portfolio composition
#and wants to know the probability of being able to make donation at end of life

wealth =2e6
equity_pct=.4                                       #percent of portfolio in equities rebalaced annually
withdraw=2e5                                        #annual withdrawal for living expence
donation=1e6                                        #wants to donate 1 mill at end of life
years = 10

In [22]:
#sample TLT and IVV returns from the same day in dataset to preserve negative correlation
def wealth_planning(wealth,equity_pct,withdraw,returns,years):
    #initial
    equity=wealth*equity_pct
    bonds=wealth*(1-equity_pct)
    yr_end=0
    for j in range(years):
        for i in range(253):  #for each business day in the year
            sample=returns.sample()                     #sampling IVV and TLT returns from the same day
            equity_rtn=1+sample.iloc[0,0]
            bond_rtn=1+sample.iloc[0,1]
            equity*=equity_rtn
            bonds*=bond_rtn 
        yr_end=equity+bonds-withdraw
        equity=yr_end*equity_pct       #rebalancing yearly
        bonds=yr_end*(1-equity_pct)
    return (yr_end)

In [25]:
#step 2: sim

def simulation(num):
    sum=0
    n=0
    sims=[]
    for i in range(num):
        sim=wealth_planning(wealth,equity_pct,withdraw,returns,years)
        sims.append(sim)
        sum+=sim
        n+=1
        if n %10==0:
            print(n)
    return(sims)

In [26]:
sims=pd.DataFrame(simulation(1000))

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000


In [27]:
import scipy.stats as stats
stats.skew(sims)

array([1.0927695])

In [28]:
stats.kurtosis(sims)

array([1.93827693])

In [29]:
sims[sims>1e6].count()/1000

0    0.743
dtype: float64

In [30]:
sims.describe()

,0
count,1.000000e+03
mean,1.755956e+06
std,1.112112e+06
min,-5.987280e+05
25%,9.823565e+05
50%,1.581905e+06
75%,2.347320e+06
max,7.159911e+06
